<img src="logo.png" height="200" width="900"> 

#  Сбор данных: грязная работа вашими руками 

Пришло время самостоятельно написать парсер! Мы будем собирать данные [о ценах на книги.](http://books.toscrape.com)


> __ВНИМАНИЕ!__ Почему-то у Coursera после обновления на этом задании перестал работать грейдер. В связи с этим все ответы, которые вы получите при написании кода, придется перенести в тест, который идёт в курсе сразу после этого задания. 

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm

#библиотека для отправки запросов
from requests import get

#для генерации случайного сочетания операц. системы, браузера и т.д.
from fake_useragent import UserAgent

#для поисков тегов, классов и прочих элементов веб страниц
from bs4 import BeautifulSoup

In [2]:
# фиктивная проверка

Прогуляйтесь на сайт http://books.toscrape.com/ и изучите его структуру.  


# 1. Сбор ссылок на книги

Напишите функцию `get_soup`, которая по ссылке возвращает html-разметку страницы в формате `bs4` 

In [3]:
def get_page_soup(url_link):
    response = get(url_link,headers={'User-Agent':UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [4]:
main_url = 'http://books.toscrape.com/catalogue/'
page_number = 5

soup = get_page_soup(main_url + f'page-{page_number}.html')
soup.find_all('a')[0].text

'Books to Scrape'

Напишите функцию `get_books_links`, которая находит в html-разметке страницы ссылки на странички с отдельными книгами. 

In [5]:
def get_book_links(page_soup):
    links = soup.find_all('article', {'class': 'product_pod'})
    links=['http://books.toscrape.com/catalogue/'+  link.a.get("href") for link in links]
    return links

С помощью цикла соберите в лист `book_links` первые 200 книг.

In [6]:
book_links = []
for page in range(1, 11):
    soup = get_page_soup(main_url + f'page-{page}.html')
    book_links.extend(get_book_links(soup))

# 2. Сбор информации о книгах 

Напишите несколько небольших функций, которые собирают различные данные об одной книге, необходимые для ответов на вопросы ниже. Информацию о книге собирайте в виде словаря вида 

```
{ 'name': 'Преступление и наказание', 'rating': 1, 'description': 'ужасно депрессивная книга', ... }

```

In [7]:
def books_info(soup):
    rating_cat = {
        'One': 1,
        'Two': 2,
        'Three': 3, 
        'Four': 4,
        'Five': 5
            } 
    rating = rating_cat[soup.find_all('p')[2].get('class')[1]]
    
    name = soup.find('div', class_="col-sm-6 product_main").find('h1').text
    
    if len(soup.find_all('div', class_='sub-header', id = 'product_description')) == 0:
        descr = None
    else:
        descr = soup.find_all('p')[3].text
    
    tax = float(soup.find('table',class_="table table-striped").find_all('td')[4].text[1:])

    price = float(soup.find('table',class_="table table-striped").find_all('td')[2].text[1:])
    
    return  {
        'name': name,
        'rating': rating,
        'price': price,
        'tax': tax,
        'description': descr,

           }

Пройдите циклом по всем сыслкам из списка `book_links` и соберите данные о книгах в вектор `book_info`. 

In [8]:
book_info = []
for book in tqdm(book_links):
    response = get(book)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    book_info.append(books_info(soup))

  0%|          | 0/200 [00:00<?, ?it/s]

Превратим вектор из информации в полноценную таблицу с данными. 

In [9]:
book_info_df = pd.DataFrame(book_info)
print(book_info_df.shape)
book_info_df.head()

(200, 5)


,name,rating,price,tax,description
0,A Light in the Attic,3,51.77,0.0,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,1,53.74,0.0,"""Erotic and absorbing...Written with starling ..."
2,Soumission,1,50.10,0.0,"Dans une France assez proche de la nôtre, un h..."
3,Sharp Objects,4,47.82,0.0,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,5,54.23,0.0,From a renowned historian comes a groundbreaki...


Теперь, когда все данные собраны, настало время ответить на несколько вопросов:

- У скольких книг отсутствует описание? Ответ на этот вопрос вбейте в тест, который идёт после лабораторной работы. 

In [10]:
ans1 = book_info_df.description.isna().sum()
ans1

1

- Сколько раз в данных встречается налог, больший нуля?  Ответ на этот вопрос вбейте в тест, который идёт после лабораторной работы. 

In [11]:
ans2 =(book_info_df.tax > 0).sum()
ans2

0

- Сколько раз рейтинг книги составлял пять звезд?  Ответ на этот вопрос вбейте в тест, который идёт после лабораторной работы. 

In [12]:
ans3 = (book_info_df.rating == 5).sum()
ans3

37

- Какова средняя цена книг (без учета налога)?  Ответ на этот вопрос вбейте в тест, который идёт после лабораторной работы. 

In [13]:
ans4 = (book_info_df.price).mean() 
ans4

34.79624999999999